In [1]:
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
from datetime import timedelta

In [2]:
mid_night = datetime.datetime.strptime('23:59:59', '%H:%M:%S').time()
noon = datetime.datetime.strptime('12:00:00', '%H:%M:%S').time()
def str_to_time(x, start_time):
    if start_time.time() > noon: 
        if noon < datetime.datetime.strptime(x[:-4], '%H:%M:%S').time() < mid_night:
            return datetime.datetime.strptime(str(start_time.date()) + " " + x[:-4], '%Y-%m-%d %H:%M:%S')
        else:
            return datetime.datetime.strptime(str(start_time.date()) + " " + x[:-4], '%Y-%m-%d %H:%M:%S') + timedelta(days=1)
    else:
        return datetime.datetime.strptime(str(start_time.date()) + " " + x[:-4], '%Y-%m-%d %H:%M:%S')

In [3]:
def round_time(x):
    timestamp = datetime.datetime.strptime(x[0:10] + " " + x[11:-6], '%Y-%m-%d %H:%M:%S.%f')
    if timestamp.time().microsecond >= 500000:
        return datetime.datetime.strptime(x[0:10] + " " + x[11:-10], '%Y-%m-%d %H:%M:%S') + timedelta(seconds=1)
    else:
        return datetime.datetime.strptime(x[0:10] + " " + x[11:-10], '%Y-%m-%d %H:%M:%S')
    
    

In [4]:
rootdir = '../Sleep Data/'
sleep_profiles = []
xethru_recordings = []

for subdir, dirs, files in os.walk(rootdir):
    subdir_split = subdir.split('/')
    subdir_und = subdir_split[-1].split('_')
    
    current_xethru_recording = pd.DataFrame(columns=["time", "frame_count", "state", "rpm", "object_distance", 
                                        "signal_quality", "movement_slow", "movement_fast"])
    
    isFirstFile = True
    
    CSVCount = 0
            
    for file in files:        
        if (file == 'Sleep profile.txt' ) & ('PSG_analysis_manual' in subdir_split):
            start_time = ''
            with open(os.path.join(subdir, file)) as f:
                head = [next(f) for x in range(2)]
                start_time = datetime.datetime.strptime(head[1][12:-1], '%d.%m.%Y %H:%M:%S')
                
            sleep_profile = pd.read_csv(os.path.join(subdir, file), sep=";", skiprows=7, header=None)
            sleep_profile.columns = ["time", "stage"]
            sleep_profile['time'] = sleep_profile['time'].apply(lambda x: str_to_time(x, start_time))
            sleep_profiles.append(sleep_profile)
        
        file_split = file.split('_')
        
        if 'xethru' in file_split and 'sleep' in file_split and 'nattbord' in subdir_und:
            numCSVs = len(glob.glob1(subdir, "*.csv"))
            
            CSVCount = CSVCount + 1
            
            xethru_recording = pd.read_csv(os.path.join(subdir, file), sep=";", skiprows=1, header=None)

            xethru_recording.columns = ["time", "frame_count", "state", "rpm", "object_distance", 
                                        "signal_quality", "movement_slow", "movement_fast"]
            
            xethru_recording['time'] = xethru_recording['time'].apply(lambda x: round_time(x))
            
            start_timestamp = sleep_profiles[-1]['time'].iloc[0]
            end_timestamp = sleep_profiles[-1]['time'].iloc[-1]
            
#             while isFirstFile and xethru_recording['time'].iloc[0] >= start_timestamp:                
#                 sleep_profiles[-1] = sleep_profiles[-1].iloc[1:]
#                 start_timestamp = sleep_profiles[-1]['time'].iloc[0]
                
                               
#             while (numCSVs == CSVCount) and xethru_recording['time'].iloc[-1] <= end_timestamp:                
#                 sleep_profiles[-1] = sleep_profiles[-1].iloc[:1]                
#                 end_timestamp = sleep_profiles[-1]['time'].iloc[-1]
                                               
            current_xethru_recording = current_xethru_recording.append(xethru_recording)
        
            if numCSVs == CSVCount:
                start_time = current_xethru_recording['time'].iloc[0]
                end_time = current_xethru_recording['time'].iloc[-1]
                
                sleep_profiles[-1] = sleep_profiles[-1][sleep_profiles[-1]['time'].map(lambda x: start_time <= x <= end_time)]
                                    
                        
            isFirstFile = False            
    
    if len(current_xethru_recording) > 0 and 'nattbord' in subdir_und:
#         current_xethru_recording = current_xethru_recording.drop_duplicates('time', keep='first')
        for x in range(1, current_xethru_recording.shape[0]):
            if (current_xethru_recording['time'].iloc[x]) != (current_xethru_recording['time'].iloc[x-1] + timedelta(seconds=1)):
                time_adjusted = current_xethru_recording['time'].iloc[x-1] + timedelta(seconds=1)
                current_xethru_recording['time'].iloc[x] = time_adjusted

#         for x in range(1, current_xethru_recording.shape[0]):
#             if (current_xethru_recording['time'].iloc[x]) != (current_xethru_recording['time'].iloc[x-1] + timedelta(seconds=1)):
#                 print(subdir)
#                 print('t1', current_xethru_recording['time'].iloc[x-1])
#                 print('t2', str(current_xethru_recording['time'].iloc[x]) + '\n')

        start_timestamp = sleep_profiles[-1]['time'].iloc[0]
        end_timestamp = sleep_profiles[-1]['time'].iloc[-1] + timedelta(seconds=29)
        current_xethru_recording = current_xethru_recording[current_xethru_recording['time'].map(lambda x: start_timestamp <= x <= end_timestamp)]        
        xethru_recordings.append(current_xethru_recording)

/home/isuru/.local/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
len(xethru_recordings[4])/30

1030.0

In [16]:
all_xethru_recordings = pd.concat(xethru_recordings, ignore_index=True)
all_sleep_profiles = pd.concat(sleep_profiles, ignore_index=True)

In [17]:
labels = np.array(all_sleep_profiles['stage'].to_numpy())

In [7]:
# allLabels = [labels[allData.index(data)] for data in allData if 0.0 not in data]
# allData = [data for data in allData if 0.0 not in data]

In [8]:
# print('Data: ', len(allData))
# print('Labels: ', len(allLabels))

In [18]:
xethru_numpy = np.array(all_xethru_recordings['rpm'])

In [19]:
data = np.array(np.split(xethru_numpy, len(xethru_numpy)/30))

In [116]:
# data = data.tolist()
# allLabels_new = [labels[data.index(element)] for element in data if 0.0 not in element]
# allData_new = [element for element in data if 0.0 not in element]

In [117]:
# allData_new = np.array(allData_new)
# allLabels_new = np.array(allLabels_new)

In [16]:
# np.place(allLabels_new, allLabels_new==' Wake', [0])
# np.place(allLabels_new, allLabels_new==' N1', [1])
# np.place(allLabels_new, allLabels_new==' N2', [2])
# np.place(allLabels_new, allLabels_new==' N3', [3])
# np.place(allLabels_new, allLabels_new==' REM', [4])

In [ ]:
# np.place(labels, labels==' Wake', [0])
# np.place(labels, labels==' N1', [1])
# np.place(labels, labels==' N2', [2])
# np.place(labels, labels==' N3', [3])
# np.place(labels, labels==' REM', [4])

In [ ]:
# allLabels = allLabels.astype('int32')

In [ ]:
# np.savetxt('allLabels.csv', allLabels, delimiter=',', fmt='%s')

In [ ]:
# np.save('allData.npy', allData)

In [ ]:
# allData = np.load('allData.npy', allow_pickle=True)

In [ ]:
# allLabels = np.genfromtxt('allLabels.csv', delimiter=',').astype('int32')

In [20]:
print('data: ', len(labels))
print('labels: ', len(data))

data:  39907
labels:  39907


In [21]:
labels = np.array(labels)
data = np.array(data)

In [22]:
remove = []
for i in range(len(labels)):
    if str(labels[i]) == ' A':
        remove.append(i)
    elif str(labels[i]) == ' Artefact':
        remove.append(i)
        
data_a = np.delete(data, remove, axis = 0)
labels_a = np.delete(labels, remove)

In [23]:
np.place(labels_a, labels_a==' Wake', [0])
np.place(labels_a, labels_a==' N1', [1])
np.place(labels_a, labels_a==' N2', [2])
np.place(labels_a, labels_a==' N3', [3])
np.place(labels_a, labels_a==' N4', [4])
np.place(labels_a, labels_a==' REM', [5])

In [28]:
# np.place(labels_a, labels_a==4, [5])
# np.place(labels_a, labels_a==' N4', [4])

In [29]:
labels_a = labels_a.astype('int32')

In [99]:
from modwt import modwt, modwtmra

ET = []

WT = []

count = 0

for element in data_a:
    wt = modwt(element, 'db4', 6)
    WT.append(wt)
    E = []
    for w in wt:
        E.append(np.sum(np.abs(w) ** 2))
    ET.append(E)
    count += 1
    print("E appended: ", count)

E appended:  1
E appended:  2
E appended:  3
E appended:  4
E appended:  5
E appended:  6
E appended:  7
E appended:  8
E appended:  9
E appended:  10
E appended:  11
E appended:  12
E appended:  13
E appended:  14
E appended:  15
E appended:  16
E appended:  17
E appended:  18
E appended:  19
E appended:  20
E appended:  21
E appended:  22
E appended:  23
E appended:  24
E appended:  25
E appended:  26
E appended:  27
E appended:  28
E appended:  29
E appended:  30
E appended:  31
E appended:  32
E appended:  33
E appended:  34
E appended:  35
E appended:  36
E appended:  37
E appended:  38
E appended:  39
E appended:  40
E appended:  41
E appended:  42
E appended:  43
E appended:  44
E appended:  45
E appended:  46
E appended:  47
E appended:  48
E appended:  49
E appended:  50
E appended:  51
E appended:  52
E appended:  53
E appended:  54
E appended:  55
E appended:  56
E appended:  57
E appended:  58
E appended:  59
E appended:  60
E appended:  61
E appended:  62
E appended:  63
E

E appended:  499
E appended:  500
E appended:  501
E appended:  502
E appended:  503
E appended:  504
E appended:  505
E appended:  506
E appended:  507
E appended:  508
E appended:  509
E appended:  510
E appended:  511
E appended:  512
E appended:  513
E appended:  514
E appended:  515
E appended:  516
E appended:  517
E appended:  518
E appended:  519
E appended:  520
E appended:  521
E appended:  522
E appended:  523
E appended:  524
E appended:  525
E appended:  526
E appended:  527
E appended:  528
E appended:  529
E appended:  530
E appended:  531
E appended:  532
E appended:  533
E appended:  534
E appended:  535
E appended:  536
E appended:  537
E appended:  538
E appended:  539
E appended:  540
E appended:  541
E appended:  542
E appended:  543
E appended:  544
E appended:  545
E appended:  546
E appended:  547
E appended:  548
E appended:  549
E appended:  550
E appended:  551
E appended:  552
E appended:  553
E appended:  554
E appended:  555
E appended:  556
E appended:  5

E appended:  1006
E appended:  1007
E appended:  1008
E appended:  1009
E appended:  1010
E appended:  1011
E appended:  1012
E appended:  1013
E appended:  1014
E appended:  1015
E appended:  1016
E appended:  1017
E appended:  1018
E appended:  1019
E appended:  1020
E appended:  1021
E appended:  1022
E appended:  1023
E appended:  1024
E appended:  1025
E appended:  1026
E appended:  1027
E appended:  1028
E appended:  1029
E appended:  1030
E appended:  1031
E appended:  1032
E appended:  1033
E appended:  1034
E appended:  1035
E appended:  1036
E appended:  1037
E appended:  1038
E appended:  1039
E appended:  1040
E appended:  1041
E appended:  1042
E appended:  1043
E appended:  1044
E appended:  1045
E appended:  1046
E appended:  1047
E appended:  1048
E appended:  1049
E appended:  1050
E appended:  1051
E appended:  1052
E appended:  1053
E appended:  1054
E appended:  1055
E appended:  1056
E appended:  1057
E appended:  1058
E appended:  1059
E appended:  1060
E appended

E appended:  1479
E appended:  1480
E appended:  1481
E appended:  1482
E appended:  1483
E appended:  1484
E appended:  1485
E appended:  1486
E appended:  1487
E appended:  1488
E appended:  1489
E appended:  1490
E appended:  1491
E appended:  1492
E appended:  1493
E appended:  1494
E appended:  1495
E appended:  1496
E appended:  1497
E appended:  1498
E appended:  1499
E appended:  1500
E appended:  1501
E appended:  1502
E appended:  1503
E appended:  1504
E appended:  1505
E appended:  1506
E appended:  1507
E appended:  1508
E appended:  1509
E appended:  1510
E appended:  1511
E appended:  1512
E appended:  1513
E appended:  1514
E appended:  1515
E appended:  1516
E appended:  1517
E appended:  1518
E appended:  1519
E appended:  1520
E appended:  1521
E appended:  1522
E appended:  1523
E appended:  1524
E appended:  1525
E appended:  1526
E appended:  1527
E appended:  1528
E appended:  1529
E appended:  1530
E appended:  1531
E appended:  1532
E appended:  1533
E appended

E appended:  1950
E appended:  1951
E appended:  1952
E appended:  1953
E appended:  1954
E appended:  1955
E appended:  1956
E appended:  1957
E appended:  1958
E appended:  1959
E appended:  1960
E appended:  1961
E appended:  1962
E appended:  1963
E appended:  1964
E appended:  1965
E appended:  1966
E appended:  1967
E appended:  1968
E appended:  1969
E appended:  1970
E appended:  1971
E appended:  1972
E appended:  1973
E appended:  1974
E appended:  1975
E appended:  1976
E appended:  1977
E appended:  1978
E appended:  1979
E appended:  1980
E appended:  1981
E appended:  1982
E appended:  1983
E appended:  1984
E appended:  1985
E appended:  1986
E appended:  1987
E appended:  1988
E appended:  1989
E appended:  1990
E appended:  1991
E appended:  1992
E appended:  1993
E appended:  1994
E appended:  1995
E appended:  1996
E appended:  1997
E appended:  1998
E appended:  1999
E appended:  2000
E appended:  2001
E appended:  2002
E appended:  2003
E appended:  2004
E appended

...

E appended:  37672
E appended:  37673
E appended:  37674
E appended:  37675
E appended:  37676
E appended:  37677
E appended:  37678
E appended:  37679
E appended:  37680
E appended:  37681
E appended:  37682
E appended:  37683
E appended:  37684
E appended:  37685
E appended:  37686
E appended:  37687
E appended:  37688
E appended:  37689
E appended:  37690
E appended:  37691
E appended:  37692
E appended:  37693
E appended:  37694
E appended:  37695
E appended:  37696
E appended:  37697
E appended:  37698
E appended:  37699
E appended:  37700
E appended:  37701
E appended:  37702
E appended:  37703
E appended:  37704
E appended:  37705
E appended:  37706
E appended:  37707
E appended:  37708
E appended:  37709
E appended:  37710
E appended:  37711
E appended:  37712
E appended:  37713
E appended:  37714
E appended:  37715
E appended:  37716
E appended:  37717
E appended:  37718
E appended:  37719
E appended:  37720
E appended:  37721
E appended:  37722
E appended:  37723
E appended: 

E appended:  38127
E appended:  38128
E appended:  38129
E appended:  38130
E appended:  38131
E appended:  38132
E appended:  38133
E appended:  38134
E appended:  38135
E appended:  38136
E appended:  38137
E appended:  38138
E appended:  38139
E appended:  38140
E appended:  38141
E appended:  38142
E appended:  38143
E appended:  38144
E appended:  38145
E appended:  38146
E appended:  38147
E appended:  38148
E appended:  38149
E appended:  38150
E appended:  38151
E appended:  38152
E appended:  38153
E appended:  38154
E appended:  38155
E appended:  38156
E appended:  38157
E appended:  38158
E appended:  38159
E appended:  38160
E appended:  38161
E appended:  38162
E appended:  38163
E appended:  38164
E appended:  38165
E appended:  38166
E appended:  38167
E appended:  38168
E appended:  38169
E appended:  38170
E appended:  38171
E appended:  38172
E appended:  38173
E appended:  38174
E appended:  38175
E appended:  38176
E appended:  38177
E appended:  38178
E appended: 

E appended:  38568
E appended:  38569
E appended:  38570
E appended:  38571
E appended:  38572
E appended:  38573
E appended:  38574
E appended:  38575
E appended:  38576
E appended:  38577
E appended:  38578
E appended:  38579
E appended:  38580
E appended:  38581
E appended:  38582
E appended:  38583
E appended:  38584
E appended:  38585
E appended:  38586
E appended:  38587
E appended:  38588
E appended:  38589
E appended:  38590
E appended:  38591
E appended:  38592
E appended:  38593
E appended:  38594
E appended:  38595
E appended:  38596
E appended:  38597
E appended:  38598
E appended:  38599
E appended:  38600
E appended:  38601
E appended:  38602
E appended:  38603
E appended:  38604
E appended:  38605
E appended:  38606
E appended:  38607
E appended:  38608
E appended:  38609
E appended:  38610
E appended:  38611
E appended:  38612
E appended:  38613
E appended:  38614
E appended:  38615
E appended:  38616
E appended:  38617
E appended:  38618
E appended:  38619
E appended: 

E appended:  39019
E appended:  39020
E appended:  39021
E appended:  39022
E appended:  39023
E appended:  39024
E appended:  39025
E appended:  39026
E appended:  39027
E appended:  39028
E appended:  39029
E appended:  39030
E appended:  39031
E appended:  39032
E appended:  39033
E appended:  39034
E appended:  39035
E appended:  39036
E appended:  39037
E appended:  39038
E appended:  39039
E appended:  39040
E appended:  39041
E appended:  39042
E appended:  39043
E appended:  39044
E appended:  39045
E appended:  39046
E appended:  39047
E appended:  39048
E appended:  39049
E appended:  39050
E appended:  39051
E appended:  39052
E appended:  39053
E appended:  39054
E appended:  39055
E appended:  39056
E appended:  39057
E appended:  39058
E appended:  39059
E appended:  39060
E appended:  39061
E appended:  39062
E appended:  39063
E appended:  39064
E appended:  39065
E appended:  39066
E appended:  39067
E appended:  39068
E appended:  39069
E appended:  39070
E appended: 

E appended:  39455
E appended:  39456
E appended:  39457
E appended:  39458
E appended:  39459
E appended:  39460
E appended:  39461
E appended:  39462
E appended:  39463
E appended:  39464
E appended:  39465
E appended:  39466
E appended:  39467
E appended:  39468
E appended:  39469
E appended:  39470
E appended:  39471
E appended:  39472
E appended:  39473
E appended:  39474
E appended:  39475
E appended:  39476
E appended:  39477
E appended:  39478
E appended:  39479
E appended:  39480
E appended:  39481
E appended:  39482
E appended:  39483
E appended:  39484
E appended:  39485
E appended:  39486
E appended:  39487
E appended:  39488
E appended:  39489
E appended:  39490
E appended:  39491
E appended:  39492
E appended:  39493
E appended:  39494
E appended:  39495
E appended:  39496
E appended:  39497
E appended:  39498
E appended:  39499
E appended:  39500
E appended:  39501
E appended:  39502
E appended:  39503
E appended:  39504
E appended:  39505
E appended:  39506
E appended: 

In [198]:
PET = []

for E in ET:
    PE = []
    for element in E:
        PE.append(element/np.sum(E))
    PET.append(PE)

/home/isuru/.conda/envs/maritime_surveillance/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [199]:
MT = []

for W in WT:
    M = []
    for element in W:        
        M.append(np.mean(w))
    MT.append(M)

In [200]:
StdT = []

for W in WT:
    Std = []
    for element in W:        
        Std.append(np.std(w))
    StdT.append(Std)

In [201]:
PT = []

for element in data:
    PT.append(np.max(element) - np.min(element))

In [212]:
data_all = np.concatenate((ET, PET, MT, StdT), axis = 1)

In [213]:
data_all.shape

(39865, 28)

In [100]:
# ET_np = np.array(ET)

In [116]:
WT_np = np.array(WT)

In [172]:
# WT_np_r = np.reshape(WT_np, (WT_np.shape[0], 1, WT_np.shape[1] * WT_np.shape[2]))

In [195]:
data_a_r = np.reshape(data_a, (data_a.shape[0], 1, data_a.shape[1]))

In [196]:
unique, counts = np.unique(labels_a, return_counts=True)
dict(zip(unique, counts))

{0: 4173, 1: 2145, 2: 16479, 3: 8322, 4: 136, 5: 8610}

In [174]:
# class_weight = {0:0.75, 1:1, 2:0.1, 3:0.2, 4:0.3}

In [175]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras import Input

In [176]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [215]:
# data_all_p = preprocessing.normalize(data_all)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [209]:
# ET_np_p = preprocessing.scale(ET_np)

In [210]:
# WT_np_r_p = preprocessing.scale(WT_np_r)

In [225]:
train_data, test_data, train_labels, test_labels = train_test_split(ET_np_p, labels_a, test_size=0.1, random_state=0)

In [226]:
train_data.shape

(35878, 7)

In [227]:
# Embedding
max_features = 30000
maxlen = 30
embedding_size = 128

# Convolution
kernel_size = 16
filters = 128
pool_size = 4

# LSTM
lstm_output_size = 60

# Training
batch_size = 30
epochs = 20

In [228]:
# model = Sequential()
# # model.add(Embedding(max_features, embedding_size, input_length=maxlen))
# # model.add(Dropout(0.25))
# # model.add(Conv1D(filters,
# #                  kernel_size,
# #                  padding='valid',
# #                  activation='relu',
# #                  strides=2))
# # model.add(MaxPooling1D(pool_size=pool_size))

# model.add(LSTM(units=256,
#                input_shape=train_data[0].shape, return_sequences = True))
# model.add(Dropout(0.25))
# model.add(LSTM(units=256, activation='relu',
#                input_shape=train_data[0].shape))
# # model.add(LSTM(units=256, activation='relu', input_shape=(1,7)))
# # model.add(Dropout(0.25))
# # model.add(LSTM(units=512, activation='relu'))
# model.add(Dense(6, activation='relu'))
# model.add(Activation('softmax'))

In [229]:
model = Sequential()
model.add(Dense(1024, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [230]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, train_labels,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(test_data, test_labels))

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn

y_pred = model.predict(test_data)

test_labels = test_labels.astype('int32')

confusion_matrix = confusion_matrix(test_labels, y_pred.argmax(axis=1))

df_cm = pd.DataFrame(confusion_matrix, index = ['Wake', 'N1', 'N2', 'N3', 'N4', 'REM'],
                  columns = ['Wake', 'N1', 'N2', 'N3', 'N4', 'REM'])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)